In [1]:
# importing some useful libraries
%pylab inline

import pandas as pd
import time 
import joblib

from sklearn import metrics
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from PIL import Image
rng = np.random.default_rng(12345)

Populating the interactive namespace from numpy and matplotlib


In [2]:
MODEL_FILE = "../Dumps/model-combined-URLs.sav"
SCALER_FILE = "../Dumps/scaler-combined-URLs.sav"
ENCODER_FILE = "../Dumps/encoder-combined-URLs.sav"

URL_REGEX = ("^((?P<scheme>[^:/?#]+):(?=//))?(//)?(((?P<login>[^:]+)" + 
    "(?::(?P<password>[^@]+)?)?@)?(?P<host>[^@/?#:]*)(?::(?P<port>\d+)?)?)?" + 
    "(?P<path>[^?#]*)(\?(?P<query>[^#]*))?(#(?P<fragment>.*))?")

"^(?P<url>(?P<protocol>http[s]?|ftp):\/)?\/?(?P<host>[^:\/\s]+)(?P<path>(\/\w+)*\/)(?P<file>[\w\-\.]+[^#?\s]+)(?P<query>.*)?(?P<hash>#[\w\-]+)?$"

'^(?P<url>(?P<protocol>http[s]?|ftp):\\/)?\\/?(?P<host>[^:\\/\\s]+)(?P<path>(\\/\\w+)*\\/)(?P<file>[\\w\\-\\.]+[^#?\\s]+)(?P<query>.*)?(?P<hash>#[\\w\\-]+)?$'

# Wczytywanie danych

In [3]:
df = pd.read_csv("../Data/cleaned_combined_urls.csv", index_col=0)
df.head(2)

c:\users\piotr\pycharmprojects\hackathon-2021-rtfd\venv\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,url,label
0,?guid=Windows Updates Manager,1.0
1,69.162.100.198/,1.0


In [4]:
df["url"] = df.url.astype(str)

## Czyszczenie danych

In [5]:
print(df.shape)
print(df[df.url.str.count(".") > 0].shape)

(1958842, 2)
(1958842, 2)


In [6]:
url_parse_groups = ["scheme", 4, "host", "port", "path", "query", "fragment"]
df = pd.concat(
    [df, df.url.str.extract(URL_REGEX)[url_parse_groups]],
    axis=1
)    
print(df.shape)
df.columns = ["url", "label", "scheme", 
              "user", "host", "port", 
              "path", "query", "fragment"]
df.head()

(1958842, 9)


,url,label,scheme,user,host,port,path,query,fragment
0,?guid=Windows Updates Manager,1.0,NaN,NaN,,NaN,,guid=Windows Updates Manager,NaN
1,69.162.100.198/,1.0,NaN,NaN,69.162.100.198,NaN,/,NaN,NaN
2,babicz123.ddns.net/,1.0,NaN,NaN,babicz123.ddns.net,NaN,/,NaN,NaN
3,highpowerresources.com,1.0,NaN,NaN,highpowerresources.com,NaN,,NaN,NaN
4,intent.nofrillspace.com/users/web11_focus/380...,1.0,NaN,NaN,intent.nofrillspace.com,NaN,/users/web11_focus/3807/space.gif,NaN,NaN


In [7]:
df.drop(np.where(df.host.str.len() == 0)[0], inplace=True)

In [8]:
df.groupby("label").count()

,url,scheme,user,host,port,path,query,fragment
label,,,,,,,,
0.0,1725805,963664,0,1725805,182,1725805,19340,518
1.0,233036,154514,0,233036,1121,233036,10188,135


In [9]:
from pathlib import Path
df["filename"] = [Path(x).stem for x in  df["path"]]
df["fileext"] = [Path(x).suffix.replace('.', '') for x in  df["path"]]

In [10]:
df.head(2)

,url,label,scheme,user,host,port,path,query,fragment,filename,fileext
1,69.162.100.198/,1.0,NaN,NaN,69.162.100.198,NaN,/,NaN,NaN,,
2,babicz123.ddns.net/,1.0,NaN,NaN,babicz123.ddns.net,NaN,/,NaN,NaN,,


## Obliczanie metryk

### Popularność domeny

In [11]:
df['tld'] = df.host.str.split(".").apply(lambda x : x[-1])
count = df.groupby(["tld"]).count().reset_index().iloc[:, [0,1]]
count.columns = ["tld", "count"]
df = df.merge(count, on=["tld"])
df.head(10)

,url,label,scheme,user,host,port,path,query,fragment,filename,fileext,tld,count
0,69.162.100.198/,1.0,NaN,NaN,69.162.100.198,NaN,/,NaN,NaN,,,198,11
1,100.42.48.198/~fruiti/js/prototype/windows/the...,1.0,NaN,NaN,100.42.48.198,NaN,/~fruiti/js/prototype/windows/themes/default/f...,NaN,NaN,express,html,198,11
2,185.66.9.198/module/09e2a5987534a3f12a01504e28...,1.0,NaN,NaN,185.66.9.198,NaN,/module/09e2a5987534a3f12a01504e2823f826/,NaN,NaN,09e2a5987534a3f12a01504e2823f826,,198,11
3,193.238.152.198/OeeC,1.0,NaN,NaN,193.238.152.198,NaN,/OeeC,NaN,NaN,OeeC,,198,11
4,http://209.150.104.198/horror/julieshiel/,0.0,http,NaN,209.150.104.198,NaN,/horror/julieshiel/,NaN,NaN,julieshiel,,198,11
5,216.22.34.198/uploadform/server/php/files/GD/i...,1.0,NaN,NaN,216.22.34.198,NaN,/uploadform/server/php/files/GD/index.php,NaN,NaN,index,php,198,11
6,37.187.65.198/Chrome_Update_2014.exe,1.0,NaN,NaN,37.187.65.198,NaN,/Chrome_Update_2014.exe,NaN,NaN,Chrome_Update_2014,exe,198,11
7,5.200.52.198/flex/3435dde3r.exe,1.0,NaN,NaN,5.200.52.198,NaN,/flex/3435dde3r.exe,NaN,NaN,3435dde3r,exe,198,11
8,91.239.24.198:6892,1.0,NaN,NaN,91.239.24.198,6892,,NaN,NaN,,,198,11
9,91.239.25.198:6892,1.0,NaN,NaN,91.239.25.198,6892,,NaN,NaN,,,198,11


In [12]:
"""This module is used to calculate Shannon entropy of objects"""
import re
from math import log2


prop_of_letters = {
    'e' : 0.1202, 't' : 0.0910, 'a' : 0.0812, 'o' : 0.0768,
    'i' : 0.0731, 'n' : 0.0695, 's' : 0.0628, 'r' : 0.0602,
    'h' : 0.0592, 'd' : 0.0432, 'l' : 0.0398, 'u' : 0.0288,
    'c' : 0.0271, 'm' : 0.0261, 'f' : 0.0230, 'y' : 0.0211,
    'w' : 0.0209, 'g' : 0.0203, 'p' : 0.0182, 'b' : 0.0149,
    'v' : 0.0111, 'k' : 0.0069, 'x' : 0.0017, 'q' : 0.0011,
    'j' : 0.0010, 'z' : 0.0007}

cleaner = re.compile('[^a-z]+')

def clean(test):
    return cleaner.sub('', test)

def entropy(text):
    ent = 0.0
    for letter in clean(text):
        ent += prop_of_letters[letter] * log2(prop_of_letters[letter])
    return ent

In [155]:
def character_continiuity_rate(text):
    digits  = max(map(len,re.findall("\d+", text)), default = 0)
    chars = max(map(len,re.findall("[a-z]+", text)), default = 0)
    special_chars = max(map(len,re.findall("[.\+\-\,\&\/]+", text)), default = 0)
    return (digits + chars + special_chars) / len(text)

### Pozostałe

In [165]:
def calculate_metrics(df_):
    ##### Scheme
    scheme = df_.scheme
    scheme = scheme.fillna("n")
    scheme = scheme.replace({
        "[nh]+t+p+s+": 0, 
        "[nh]+t+p*(?![a-z]*s)" : 1,
        "n": 0.5, 
        "none": 0.5,
        "\w+" : 0.5
    }, regex=True)
    scheme = scheme.astype('float64')    
    
    #### URL
    url_len = df_.url.str.len()
    
    #### Query
    
    
    #### Host
    host = df_.host
    host_len = host.str.len()
    host_subdomains_count = host.str.count(".")
    host_subdomains_mean_len = (host_len - host_subdomains_count) / host_subdomains_count
    host_digit = host.str.count("\d")
    host_nspecial = host.str.count("([^A-Za-z\d\s])")
    host_first_len = host.str.split(".", n=1, expand=True)[0].str.len() / host_len
    host_digit_letter_count = host.str.count("[A-Za-z]\d|\d[A-Za-z]")
    host_has_a = host.str.count("@")
    host_pref_suf_number = host.str.count("[-_]")
    host_is_ipv4 = host.str.count("(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)" + 
                                  "(\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}")
    host_entropy = np.array([entropy(h) for h in host.to_numpy()])
    
    
    #### Path
    path = df_.path
    path_len = path.str.len()
    path_digit = path.str.count("\d")
    
    #### Filename
    filename = df_.filename
    filename_len = filename.str.len()
    filename_digit = filename.str.count("\d")
    
    #### File extension
    df_.loc[df_["fileext"].str.len() > 4,"fileext"] = ''
    fileext = df_.fileext
    fileext_exe = fileext.str.contains("exe|jar|run", na=False)
    fileext_digit= filename.str.count("\d")
    fileext_len = filename.str.len()
    
    #### Cross 
    path_url_ratio = path_len / url_len
    host_url_ratio = host_len / url_len
    host_path_ratio = host_len / path_len
    
    ccr = host.apply(lambda x: character_continiuity_rate(x))
    
    return np.array([
        host_len, host_subdomains_count, host_subdomains_mean_len, 
        host_digit, host_nspecial, host_first_len, host_digit_letter_count, 
        host_has_a, host_pref_suf_number, host_is_ipv4, host_entropy, path_len,
        url_len, path_url_ratio, host_url_ratio, path_digit, filename_len, filename_digit,
        fileext_exe, fileext_digit, fileext_len, ccr
    ]).T

In [154]:
character_continiuity_rate("a1ego.com")

0.6

In [40]:
df.loc[df["fileext"].str.len() > 4,"fileext"] = 0
df.loc[df["fileext"].str.contains("\d|\+|\-"),"fileext"] = 0
pd.get_dummies(df.fileext)

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [74]:
df[["url", "scheme"]].groupby("scheme").count()

,url
scheme,
http,1
ftp,207
gopher,18
hXXp,24
hXXps,2
htpp,1
http,1106003
https,11888
https.portal.apple.com.idmswebauth.login.html.appidkey.05c7e09b5896b0334b3af1139274f266b2hxxp,1


# Trenowanie modelu

In [166]:
sc = StandardScaler()
dane = df
label_distribution = df.groupby("label").count()
label_distribution

,url,scheme,user,host,port,path,query,fragment,filename,fileext,tld,count
label,,,,,,,,,,,,
0.0,1725805,963664,0,1725805,182,1725805,19340,518,1725805,1725805,1725805,1725805
1.0,233036,154514,0,233036,1121,233036,10188,135,233036,233036,233036,233036


In [167]:
df[df["query"].notna()]

,url,label,scheme,user,host,port,path,query,fragment,filename,fileext,tld,count
12,'64835108.net/images/?us.battle.net/login/en/r...,1.0,NaN,NaN,'64835108.net,NaN,/images/,us.battle.net/login/en/ref=http\%3A\%2F\%2Fus....,NaN,images,,net,90987
15,'antispam5.rilinfo.net/Message?action=Detail&a...,1.0,NaN,NaN,'antispam5.rilinfo.net,NaN,/Message,action=Detail&amp;mid=219276427&amp;h=afdd9b90...,NaN,Message,,net,90987
18,'bulletpens.net/OScommerce/No/loginAction.acti...,1.0,NaN,NaN,'bulletpens.net,NaN,/OScommerce/No/loginAction.action.php,cmd=verification&appId=ABONNESNEUF&api=\%3C?ec...,NaN,loginAction.action,php,net,90987
19,'chao-yue.net/images/?us.battle.net/login/en/?...,1.0,NaN,NaN,'chao-yue.net,NaN,/images/,us.battle.net/login/en/?ref=http\%3A\%2F\%2Fus...,NaN,images,,net,90987
20,'cielopremiosnet.site90.net/xx/cadastro.html?i...,1.0,NaN,NaN,'cielopremiosnet.site90.net,NaN,/xx/cadastro.html,ios\%20autenticado&amp;pbx=1&amp;oq=&amp;aq=&a...,NaN,cadastro,html,net,90987
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957739,netsync2/post.php?filename=,1.0,NaN,NaN,netsync2,NaN,/post.php,filename=,NaN,post,php,netsync2,1
1957803,http://www.oyster.net.ck/about/index.php?about...,0.0,http,NaN,www.oyster.net.ck,NaN,/about/index.php,about=domain,NaN,index,php,ck,7
1957826,products/add_user.php?name=,1.0,NaN,NaN,products,NaN,/add_user.php,name=,NaN,add_user,php,products,1
1957827,qaz.MACLARENREPLACEMENTSTROLLERBUGGYPUSHCHAIRP...,1.0,NaN,NaN,qaz.MACLARENREPLACEMENTSTROLLERBUGGYPUSHCHAIRP...,NaN,/,q=zn_QMvXcJwDQDoPGMvrESLtEMU3QA0KK2OH_76yyEoH9...,NaN,,,UK,1


In [168]:
balanced_df_half_size = 130000
good_indices = rng.integers(0, label_distribution.url[0]-1, balanced_df_half_size)
bad_indices = rng.integers(0, label_distribution.url[1]-1, balanced_df_half_size)

balanced_df = pd.concat(
    [dane[dane.label == 0].iloc[good_indices],
     dane[dane.label == 1].iloc[bad_indices]])
le = preprocessing.LabelEncoder()
le.fit(balanced_df.label)
balanced_df["enc"] = le.transform(balanced_df.label)
balanced_df

,url,label,scheme,user,host,port,path,query,fragment,filename,fileext,tld,count,enc
787321,http://www.deleoclaytile.com/,0.0,http,NaN,www.deleoclaytile.com,NaN,/,NaN,NaN,,,com,1186983,0
814547,http://www.effectrode.com/,0.0,http,NaN,www.effectrode.com,NaN,/,NaN,NaN,,,com,1186983,0
1390111,http://www.mnfathers.org/,0.0,http,NaN,www.mnfathers.org,NaN,/,NaN,NaN,,,org,158316,0
853812,http://www.fraitagengineering.com,0.0,http,NaN,www.fraitagengineering.com,NaN,,NaN,NaN,,,com,1186983,0
1233622,http://www.visionsfantastic.com/,0.0,http,NaN,www.visionsfantastic.com,NaN,/,NaN,NaN,,,com,1186983,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59649,http://www.fidelityglocourier.net,1.0,http,NaN,www.fidelityglocourier.net,NaN,,NaN,NaN,,,net,90987,1
730121,http://www.capitalonebk.com,1.0,http,NaN,www.capitalonebk.com,NaN,,NaN,NaN,,,com,1186983,1
1807132,https://www.expressbolt.ca,1.0,https,NaN,www.expressbolt.ca,NaN,,NaN,NaN,,,ca,22067,1
750689,http://www.citibnkn.com,1.0,http,NaN,www.citibnkn.com,NaN,,NaN,NaN,,,com,1186983,1


In [169]:
features = calculate_metrics(balanced_df)
features

array([[21.        , 21.        ,  0.        , ...,  0.        ,
         0.        ,  0.66666667],
       [18.        , 18.        ,  0.        , ...,  0.        ,
         0.        ,  0.61111111],
       [17.        , 17.        ,  0.        , ...,  0.        ,
         0.        ,  0.58823529],
       ...,
       [18.        , 18.        ,  0.        , ...,  0.        ,
         0.        ,  0.66666667],
       [16.        , 16.        ,  0.        , ...,  0.        ,
         0.        ,  0.5625    ],
       [19.        , 19.        ,  0.        , ...,  0.        ,
         0.        ,  0.36842105]])

In [170]:
trainX, testX, trainY, testY = train_test_split(features, balanced_df.enc, test_size=0.2)
trainX = sc.fit_transform(trainX)
testX = sc.transform(testX)

# Wektory wspierające

In [30]:
from sklearn import svm

clf = svm.SVC()
clf.fit(trainX, trainY)

[LibSVM]

SVC(verbose=True)

In [31]:
clf.score(testX, testY)

0.6980833333333333

In [116]:
clf.score(testX[testY == 1], testY[testY == 1]) #Dane dobre = 1

AttributeError: 'function' object has no attribute 'score'

In [115]:
clf.score(testX[testY == 0], testY[testY == 0]) # Dane złe = 1

0.8086092715231789

# Drzewa losowe

In [171]:
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier(max_depth=6, random_state=0, min_samples_leaf=2)
clf2.fit(trainX, trainY)

RandomForestClassifier(max_depth=6, min_samples_leaf=2, random_state=0)

In [172]:
clf2.score(testX, testY)

0.8527884615384616

In [173]:
clf2.feature_importances_

array([2.32296295e-02, 1.82399620e-02, 0.00000000e+00, 1.46299769e-02,
       6.10416642e-02, 1.06478092e-01, 4.45308701e-03, 0.00000000e+00,
       3.01498566e-02, 6.88379823e-04, 8.38738205e-03, 1.41981134e-01,
       1.14461257e-01, 2.09107294e-01, 1.29765738e-01, 7.77839320e-03,
       4.69241013e-02, 3.00345891e-03, 5.25261179e-05, 2.29152203e-03,
       3.44242461e-02, 4.29122997e-02])

# Testowanie klasyfikatorów

In [103]:
def transform(url):
    df_ = pd.DataFrame([url], columns = ['url'])
    df_ = pd.concat(
        [df_, df_.url.str.extract(URL_REGEX)[url_parse_groups]],
        axis=1
    )
    
    df_.columns = ["url", "scheme", 
                  "user", "host", "port", 
                  "path", "query", "fragment"]
    
    df_["filename"] = [Path(x).stem for x in  df_["path"]]
    df_["fileext"] = [Path(x).suffix.replace('.', '') for x in  df_["path"]]
    
    return df_    

In [104]:
transform("https://allegro.pl")

,url,scheme,user,host,port,path,query,fragment,filename,fileext
0,https://allegro.pl,https,NaN,allegro.pl,NaN,,NaN,NaN,,


In [105]:
df.iloc[0:1000,:].apply(lambda x: str(x), axis=1)

0      url          69.162.100.198/\nlabel           ...
1      url         100.42.48.198/~fruiti/js/prototype...
2      url         185.66.9.198/module/09e2a5987534a3...
3      url         193.238.152.198/OeeC\nlabel       ...
4      url         http://209.150.104.198/horror/juli...
                             ...                        
995    url         http://acdk.sourceforge.net/\nlabe...
996    url         aceconomy.net\nlabel              ...
997    url         acehjobs.net\nlabel               ...
998    url         acestream.net\nlabel              ...
999    url         acestreamsearch.net\nlabel        ...
Length: 1000, dtype: object

In [108]:
balanced_df["pred"] = clf2.predict(sc.transform(features))

In [109]:
balanced_df[(balanced_df["enc"] == 1) & (balanced_df["pred"] == 0)] # Bad predicted as good

,url,label,scheme,user,host,port,path,query,fragment,filename,fileext,tld,count,enc,pred
1438175,poszukiwania24.eu,1.0,NaN,NaN,poszukiwania24.eu,NaN,,NaN,NaN,,,eu,5098,1,0
1905129,amblxij.chromeupdate.top/,1.0,NaN,NaN,amblxij.chromeupdate.top,NaN,/,NaN,NaN,,,top,1016,1,0
1905759,pulsar-club.top,1.0,NaN,NaN,pulsar-club.top,NaN,,NaN,NaN,,,top,1016,1,0
425068,http://natwestbankgroups.com,1.0,http,NaN,natwestbankgroups.com,NaN,,NaN,NaN,,,com,1186983,1,0
1701378,997policyjka.pl,1.0,NaN,NaN,997policyjka.pl,NaN,,NaN,NaN,,,pl,8927,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845301,http://www.firstkapital.com/,1.0,http,NaN,www.firstkapital.com,NaN,/,NaN,NaN,,,com,1186983,1,0
1580582,http://www.dianavocals.co.uk/as/fbd248c6b36383...,1.0,http,NaN,www.dianavocals.co.uk,NaN,/as/fbd248c6b36383ebcca51447f86c1f75/,NaN,NaN,fbd248c6b36383ebcca51447f86c1f75,,uk,71878,1,0
1928613,nonony.host,1.0,NaN,NaN,nonony.host,NaN,,NaN,NaN,,,host,167,1,0
1359181,http://www.electronicdrumpad.org/artefect.net/ll/,1.0,http,NaN,www.electronicdrumpad.org,NaN,/artefect.net/ll/,NaN,NaN,ll,,org,158316,1,0


In [110]:
balanced_df[(balanced_df["enc"] == 0) & (balanced_df["pred"] == 1)] # Good predicted as bad

,url,label,scheme,user,host,port,path,query,fragment,filename,fileext,tld,count,enc,pred
320791,icemtour.com/turkeyhotels/queen-elizabeth-hote...,0.0,NaN,NaN,icemtour.com,NaN,/turkeyhotels/queen-elizabeth-hotel-kemer-anta...,NaN,NaN,queen-elizabeth-hotel-kemer-antalya_turkey,html,com,1186983,0,1
634601,http://www.abalos-herreros.com,0.0,http,NaN,www.abalos-herreros.com,NaN,,NaN,NaN,,,com,1186983,0,1
1163029,http://www.staufferv8.com,0.0,http,NaN,www.staufferv8.com,NaN,,NaN,NaN,,,com,1186983,0,1
958781,http://www.lakewashingtondermatology.com,0.0,http,NaN,www.lakewashingtondermatology.com,NaN,,NaN,NaN,,,com,1186983,0,1
1348153,http://www.childcareinhealthcare.org,0.0,http,NaN,www.childcareinhealthcare.org,NaN,,NaN,NaN,,,org,158316,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297644,gurganus.org/ourfamily/location.cfm/state/CT/c...,0.0,NaN,NaN,gurganus.org,NaN,/ourfamily/location.cfm/state/CT/county/New%20...,NaN,NaN,8402,,org,158316,0,1
301451,http://halut.tripod.com,0.0,http,NaN,halut.tripod.com,NaN,,NaN,NaN,,,com,1186983,0,1
1200961,http://www.tndhairaccessory.com,0.0,http,NaN,www.tndhairaccessory.com,NaN,,NaN,NaN,,,com,1186983,0,1
946281,http://www.kcdev.com,0.0,http,NaN,www.kcdev.com,NaN,,NaN,NaN,,,com,1186983,0,1


# Podsumowanie
Model w obecnym stanie jest w stanie rozpoznać proste podmiany i działa szybko, jak dla mnie nadaje się do wrzucenia.

In [111]:
clf2.predict(sc.transform(calculate_metrics(transform("https://allegro.pl"))))[0]

0

In [113]:
clf2.predict(sc.transform(calculate_metrics(transform("https://a11egro.pl"))))[0]

0

# Zapisanie modelu

In [45]:
import joblib

joblib.dump(clf, MODEL_FILE) # SVM
joblib.dump(sc, SCALER_FILE) # StandardScaler
joblib.dump(le, ENCODER_FILE) # LabelEncoder

['../Dumps/encoder.sav']